# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# Name : Omer hazan
# ID : 312473374

# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_classif

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

c:\Users\Omer\OneDrive - Holon Institute of Technology\HIT\Second_Year\Third_Semester\Machine Learning\Ex\assignment3-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
# YOUR CODE HERE
df_train.info() # no missing data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   story   753 non-null    object
 1   gender  753 non-null    object
dtypes: object(2)
memory usage: 11.9+ KB


In [12]:
# YOUR CODE HERE

### Check for duplicates

# df_train.info()
# len(df_train.story.unique())
# df_train.iloc[752]

# df = df_train.duplicated(subset=['story'], keep='first')

# df = pd.DataFrame(df_train.drop_duplicates(subset=['story'], keep="first"))
# df.info()
# df
# df_train.compare(df)

In [13]:
# Text Preprocessing:
def preprocess_story_text(text):
    old_text = text
    text = re.sub(r'\W+', ' ', text)     # Remove special characters
    text = re.sub(r'\d+', ' ', text)     # Remove numbers
    text = re.sub(r'\s\w\s', ' ', text)  # Cleanup single letters ex: "כ-3" will be "כ" after above
    # text = re.sub(r'\s+', ' ', text)   # Remove extra spacing

    # if len(old_text) != len(text):
    #     print(f"{len(old_text) - len(text)}")
    #     print(f"{old_text}\n, {text}\n\n")

    return text

df_train['story'] = df_train['story'].apply(preprocess_story_text)
df_test['story'] = df_test['story'].apply(preprocess_story_text)

In [14]:
# Text Preprocessing:
def preprocess_gender_text(text):
    text = re.sub(r'm', '1', text)     
    text = re.sub(r'f', '0', text)     

    # text = re.sub(r'm', '0', text)     
    # text = re.sub(r'f', '1', text)     
    return text

df_train['gender'] = df_train['gender'].apply(preprocess_gender_text)

In [15]:
# Creating validation set:
def train_test_split_(df_train: pd.DataFrame, split = 0.2) -> {pd.Series, pd.Series, pd.Series, pd.Series}:
    X = df_train["story"]
    y = df_train["gender"]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=split, random_state=42)

    # df_validate = pd.DataFrame({'story' : X_val.reset_index(drop=True),'gender' : y_val.reset_index(drop=True)})
    # df_training = pd.DataFrame({'story' : X_train.reset_index(drop=True),'gender' : y_train.reset_index(drop=True)})
    # return df_training, df_validate

    return X_train, X_val, y_train, y_val

In [16]:
# Finding the best Hyper-Parameters:

def hyperparameter_tuning(X_val,y_val, scaler, vectorizer):

    classifiers = {
        'Decision Tree': (DecisionTreeClassifier(), {'classifier__criterion': ['gini', 'entropy', 'log_loss'], 'classifier__max_depth': [None, 3, 5, 7, 9, 10]}),
        'K-Nearest Neighbors': (KNeighborsClassifier(), {'classifier__n_neighbors': [3, 5, 7], 'classifier__weights': ['uniform', 'distance']}),
        'K-Means': (KMeans(), {'classifier__n_clusters': [2, 3, 4, 5], 'classifier__init': ['k-means++', 'random'], 'classifier__n_init': [15]}),
        'Multinomial Naive Bayes': (MultinomialNB(), {'classifier__alpha': [0.1, 1, 10], 'classifier__fit_prior': [True, False], 'classifier__class_prior': [None, [0.3, 0.7], [0.5, 0.5]]}),
        'Linear SVM': (LinearSVC(), {'classifier__C': [0.1, 1, 10]}),
        'Multi-Layer Perceptron': (MLPClassifier(), {'classifier__hidden_layer_sizes': [(100,), (50, 50)], 'classifier__alpha': [0.0001, 0.001]})
    }
    


# vec = TfidfVectorizer(min_df = 5)
# vec = CountVectorizer(max_features=3)
# ht.tokenize()

In [ ]:
# TODO: create test preparation after tuning
def before_tuning_preparations_train(df_train:pd.DataFrame):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split_(df_train)

    # Vectorize the data:
    vec = TfidfVectorizer(mngram_range = (1,2), min_df = 5, sublinear_tf=True)
    X_train_vec = vec.fit_transform(X_train)
    X_test_vec = vec.transform(X_test)

    # Feature Selection:
    selector = SelectKBest(mutual_info_classif, k=100) 
    selector.fit(X_train_vec, y_train)
    X_train_selected = selector.transform(X_train_vec)
    X_test_selected = selector.transform(X_test_vec)

    # Scaling:
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train_selected.toarray())
    X_test = scaler.transform(X_test_selected.toarray())

    return X_train, X_test, y_train, y_test

In [17]:
# Hyper parameters tuning on validation data set
# train and test with k folds on train set
def flow():
    # Text Cleanup
    df_train['story'] = df_train['story'].apply(preprocess_story_text)
    df_test['story'] = df_test['story'].apply(preprocess_story_text)

    # Encode Gender
    df_train['gender'] = df_train['gender'].apply(preprocess_gender_text)
    df_test['gender'] = df_test['gender'].apply(preprocess_gender_text)

    # prepare data
    
    # find best modal for each

    # evaluate modal performance

    # prepare test data

    # run best modal on test

    # save result

In [18]:
def find_best_Decision_Tree(X_val: pd.Series, y_val: pd.Series):
    # vectorize
    # feature selection
    # scaling

    return None

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)